In [67]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import matplotlib.pyplot as plt
import scipy.misc
import sklearn.feature_extraction
import numpy as np
import scipy.ndimage
from skimage import measure, io
from skimage import transform
import train_UNET 
import skimage
import json
import datetime
import pickle
import tensorflow as tf


save_results = r'C:\Users\buggyr\Mosaic_Experiments\models'
load_training = r'C:\Users\buggyr\Mosaic_Experiments\data\external\Training Images 3'
load_validation = r'C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data'

In [41]:
import keras
from keras.models import *
from keras.models import Sequential
from keras.layers import Conv2D, Activation, UpSampling2D,Reshape, MaxPooling2D, Dropout, Cropping2D, merge, Input, concatenate, Conv2DTranspose
from keras.optimizers import Adadelta, Nadam, RMSprop
from keras.models import load_model



In [3]:

inputs = Input(shape = (None, None, 4))

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=-1)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=-1)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=-1)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=-1)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

up10 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv9)
conv10 = Conv2D(3, (3, 3), activation='relu',padding='same')(up10)

model = Model(inputs=[inputs], outputs=[conv10])

#model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])


model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 4) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 1184        input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, None, None, 32 9248        conv2d_1[0][0]                   
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, None, None, 32 0           conv2d_2[0][0]                   
___________________________________________________________________________________________

In [4]:
keyname = "_UNET128"
now=datetime.datetime.now()
save_file=os.path.join(save_results,now.strftime("%Y-%m-%d %H-%M")+keyname)
os.mkdir(save_file)
save_pred = os.path.join(save_file,'Epoch_Predictions')
os.mkdir(save_pred)
save_model = os.path.join(save_file,'Epoch_Models')
os.mkdir(save_model)
save_test = os.path.join(save_file,'Test_Results')
os.mkdir(save_test)

with open(os.path.join(save_file,'Model_Summary.txt'),'w') as fh:
    model.summary(print_fn=lambda x: fh.write(x + '\n'))
    
#rmsprop = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)
optimizer_func = Nadam(lr=0.00002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
loss_func='mse'

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
tbCallBack = keras.callbacks.TensorBoard(log_dir=os.path.join(save_file,'TNSR_BRD'), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = keras.callbacks.CSVLogger(os.path.join(save_file,'training.log'), separator=',', append=False)
epoch_predict = train_UNET.Save_predictions(save_pred)
model_checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(save_model,'model.{epoch:02d}-{loss:.2f}.hdf5'), monitor='loss')

model.compile(optimizer=optimizer_func,loss=loss_func)

In [ ]:
fls = len(os.listdir(load_training))
val_steps = len(os.listdir(load_validation))

                            #train_dir, patch_size, batch_size
train_generator = train_UNET.train_generator_rgb(load_training,128,32)
val_generator = train_UNET.val_generator_rgb(load_validation)

history = model.fit_generator(generator = train_generator,steps_per_epoch=3*fls,
                             epochs = 200,callbacks = [tbCallBack,csv_logger,epoch_predict,model_checkpoint],
                             validation_data = val_generator, validation_steps = val_steps, initial_epoch = 65)
print(history.history)

model.save(os.path.join(save_file,'DeMos_mod.h5'))

Epoch 66/200
4106/4107 [============================>.] - ETA: 0s - loss: 0.1679C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1003163366.jpg
(1, 192, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1003420127.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1003428081.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1005216151.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1006452823.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1007129816.jpg
(1, 224, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1007205537.jpg
(1, 240, 176, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1007320043.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1009434119.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1009692167.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosa

(1, 192, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1057210460.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1057251835.jpg
(1, 240, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1059514525.jpg
(1, 240, 176, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1060605297.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1061467336.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1061887159.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1063866640.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1065323785.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1065831604.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1066252238.jpg
(1, 128, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1066831905.jpg
(1, 240, 1

(1, 240, 160, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1112334131.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1113186657.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1115565519.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1115679311.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1117972841.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1118168796.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1118557877.jpg
(1, 240, 160, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1118752484.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1119015538.jpg
(1, 144, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1119034454.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1119418776.jpg
(1, 176, 2

(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1162606352.jpg
(1, 224, 176, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1163282319.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1163297375.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1163438830.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1164131282.jpg
(1, 208, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1164765687.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1165334998.jpg
(1, 240, 160, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1167144956.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1167662968.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1167669558.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1167908324.jpg
(1, 160, 2

(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1234206849.jpg
(1, 144, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1234293791.jpg
(1, 240, 176, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1234638761.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1234817607.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1235099089.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1235455146.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1235580648.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1235681222.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1235685934.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1236951314.jpg
(1, 240, 160, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1236964638.jpg
(1, 240, 1

...

(1, 240, 176, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1231348815.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1232148178.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1234206849.jpg
(1, 144, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1234293791.jpg
(1, 240, 176, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1234638761.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1234817607.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1235099089.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1235455146.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1235580648.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1235681222.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1235685934.jpg
(1, 160, 2

C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1056359656.jpg
(1, 240, 176, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1056415186.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1056873310.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1057089366.jpg
(1, 192, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1057210460.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1057251835.jpg
(1, 240, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1059514525.jpg
(1, 240, 176, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1060605297.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1061467336.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1061887159.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1063866640.jpg
(1, 160, 240, 4)
C:\Users\b

(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1115679311.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1117972841.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1118168796.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1118557877.jpg
(1, 240, 160, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1118752484.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1119015538.jpg
(1, 144, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1119034454.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1119418776.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1119463452.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1121053156.jpg
(1, 240, 176, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1121416483.jpg
(1, 240, 1

C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1165334998.jpg
(1, 240, 160, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1167144956.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1167662968.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1167669558.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1167908324.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1169307342.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1170586898.jpg
(1, 176, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1172514641.jpg
(1, 240, 176, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1174525839.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1174629344.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1175268298.jpg
(1, 240, 192, 4)
C:\Users\b

(1, 240, 160, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1003163366.jpg
(1, 192, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1003420127.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1003428081.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1005216151.jpg
(1, 160, 240, 4)
C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data\1006452823.jpg
(1, 176, 240, 4)
4107/4107 [==============================] - 1046s - loss: 0.1680 - val_loss: 0.2258
Epoch 70/200
3915/4107 [===========================>..] - ETA: 48s - loss: 0.1676

In [59]:
#Test Kodak
data ={}

kodak_dir = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak'
ls = len(os.listdir(kodak_dir))

kodak_generator = train_UNET.predict_generator_rgb(kodak_dir)

# k_pred = model.predict_generator(kodak_generator, steps = ls)

res_Kodak = train_UNET.predict_generator(model,kodak_generator,ls,data,5,save_test)

data['Kodak_IMGS_PSNR'] = res_Kodak[0]
data['Kodak_IMGS_SSIM'] = res_Kodak[1]
data['Kodak_AVG_PSNR']  = res_Kodak[2]
data['Kodak_AVG_SSIM']  = res_Kodak[3]

C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim01.png
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim02.png
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim03.png
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim04.png
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim05.png
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim06.png
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim07.png
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim08.png
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim09.png
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim10.png
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim11.png
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim12.png
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim13.png
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim14.pn

In [33]:
#Test McManus
McM_dir = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\McM'
ls = len(os.listdir(McM_dir))

McM_generator = train_UNET.predict_generator_rgb(McM_dir)

# k_pred = model.predict_generator(kodak_generator, steps = ls)

res_McM = train_UNET.predict_generator(model,McM_generator,ls,data,5,save_test)

data['McM_IMGS_PSNR'] = res_McM[0]
data['McM_IMGS_SSIM'] = res_McM[1]
data['McM_AVG_PSNR']  = res_McM[2]
data['McM_AVG_SSIM']  = res_McM[3]

C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\1.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\10.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\11.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\12.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\13.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\14.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\15.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\16.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\17.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\18.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\2.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\3.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\4.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\5.tif
0
255
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\6.tif
0
249
C:\Users\buggyr\Mosaic_Experime

In [35]:
#Write Results
data['Parameters'] = {
    'Loss Function': loss_func,
    'Optimizer':str(type(optimizer_func))
}
data['Training Set'] = {
    'Training Path': load_training,
}

with open(os.path.join(save_file,'results.txt'), 'w') as outfile:  
    json.dump(data, outfile,indent=4)

In [58]:
import importlib
importlib.reload(train_UNET)

<module 'train_UNET' from 'C:\\Users\\buggyr\\Mosaic_Experiments\\src\\train_UNET.py'>

In [60]:
model = keras.models.load_model(r'C:\Users\buggyr\Mosaic_Experiments\models\2017-12-19 20-07_UNET128\Epoch_Models\model.54-0.17.hdf5')

In [ ]:
save_test = os.path.join(save_file,'Test_Results')
os.mkdir(save_test)


In [ ]:
ptch_sizes = [32, 64, 128]
for ptch_size in ptch_sizes:
    print(str(ptch_size))
    
    now=datetime.datetime.now()
    save_file=os.path.join(save_results,now.strftime("%Y-%m-%d %H-%M"))
    os.mkdir(save_file)
    save_pred = os.path.join(save_file,'Epoch_Predictions')
    os.mkdir(save_pred)

    with open(os.path.join(save_file,'Model_Summary.txt'),'w') as fh:
        # Pass the file handle in as a lambda function to make it callable
        model.summary(print_fn=lambda x: fh.write(x + '\n'))

    #rmsprop = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)
    optimizer_func = Nadam(lr=0.00002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    loss_func='mse'

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
    tbCallBack = keras.callbacks.TensorBoard(log_dir=os.path.join(save_file,'TNSR_BRD'), histogram_freq=0, write_graph=True, write_images=True)
    csv_logger = keras.callbacks.CSVLogger(os.path.join(save_file,'training.log'), separator=',', append=False)
    epoch_predict = train_UNET.Save_predictions(save_pred)


    model.compile(optimizer=optimizer_func,loss=loss_func)

    fls = len(os.listdir(load_training))

                                #train_dir, patch_size, batch_size
    train_generator = train_UNET.train_generator_rgb(load_training,ptch_size,32)
    history = model.fit_generator(generator = train_generator,steps_per_epoch=3*fls,
                                 epochs = 200,callbacks = [tbCallBack,csv_logger,epoch_predict])
    print(history.history)

    model.save(os.path.join(save_file,'DeMos_mod.h5'))